# Introduction to LLM-Compressor 🗜️

[llm-compressor](https://github.com/vllm-project/llm-compressor) is the production quantization toolkit from the vLLM project. It provides a simple `oneshot` API for applying post-training quantization.

In this notebook, you'll:
1. Set up llm-compressor
2. Quantize a small model using the `oneshot` API
3. Compare the file size before and after quantization

The example included below is designed to run on CPU rather than GPU.  It's a simple demonstration only of the process involved with compression.  Real world compression for enterprise use cases will involve much larger models and require multiple GPUs.

## 0. Setup

Install dependencies. This notebook runs on CPU.

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
MODEL_ID = "Qwen/Qwen2-0.5B-Instruct"
OUTPUT_DIR = "qwen2-0.5b-W4A16"
print("Setup complete.")


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Setup complete.


In [7]:
!pip list | grep torch

torch              2.8.0

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pathlib
from transformers import AutoTokenizer, AutoModelForCausalLM


---
## 1. The Oneshot API

The `oneshot` function is llm-compressor's main interface for quantization:

```python
oneshot(
    model="model-name",           # HuggingFace model ID
    dataset="dataset-name",       # Calibration dataset
    recipe=recipe,                # Quantization configuration
    output_dir="./output",        # Where to save
    num_calibration_samples=512,  # Samples for calibration
    max_seq_length=4096,          # Sequence length
)
```

---
## 2. Understanding Recipes

A **recipe** defines how llm-compressor will quantize your model. It's a list of "modifiers" that specify the quantization algorithm and settings.

### Available Modifiers

llm-compressor provides several modifiers for different quantization approaches:

**Quantization Modifiers:**

| Modifier | Description |
|----------|-------------|
| `GPTQModifier` | GPTQ algorithm - uses calibration data to find optimal quantization values |
| `AWQModifier` | Activation-aware Weight Quantization - preserves salient weights |
| `AutoRoundModifier` | Intel's algorithm with learnable rounding/clipping |
| `QuantizationModifier` | Basic PTQ and QAT for simple use cases |

**Transform Modifiers** (for improving accuracy):

| Modifier | Description |
|----------|-------------|
| `SmoothQuantModifier` | Smooths activations before quantization - often paired with GPTQ |
| `QuIPModifier` | Hadamard rotations to reduce outliers |
| `SpinQuantModifier` | SpinQuant-style rotations to even out weight distributions |

Modifiers can be chained together. For example, applying `SmoothQuantModifier` before `GPTQModifier` can improve accuracy for W8A8 quantization.

### GPTQModifier

The most common modifier is `GPTQModifier`, which uses the GPTQ algorithm - a post-training quantization method that minimizes accuracy loss by using calibration data.

```python
from llmcompressor.modifiers.quantization import GPTQModifier

recipe = [
    GPTQModifier(
        scheme="W4A16",       # Quantization scheme
        targets="Linear",    # Which layers to quantize
        ignore=["lm_head"],  # Layers to skip
        group_size=128,      # Optional: weights per scale factor
    ),
]
```

### Quantization Schemes

The `scheme` parameter determines the bit-width for weights (W) and activations (A):

| Scheme | Weights | Activations | Size Reduction | Quality Impact |
|--------|---------|-------------|----------------|----------------|
| `W8A16` | 8-bit | 16-bit (FP16) | ~50% | Minimal |
| `W4A16` | 4-bit | 16-bit (FP16) | ~75% | Low-Moderate |
| `W8A8` | 8-bit | 8-bit | ~50% | Low |
| `W4A8` | 4-bit | 8-bit | ~75% | Moderate |

### Key Parameters

| Parameter | Purpose | Recommendation |
|-----------|---------|----------------|
| `scheme` | Bit-width for weights/activations | Start with `W4A16` for good size/quality balance |
| `targets` | Layer types to quantize | `"Linear"` targets all linear layers |
| `ignore` | Layers to keep in full precision | Always include `["lm_head"]` - the output layer is sensitive |
| `group_size` | Weights sharing a scale factor | `128` (default) works well; smaller = better quality, larger model |

### Why Our Example Configuration?

```python
GPTQModifier(
    scheme="W4A16",       # 4-bit weights give ~75% size reduction
    targets="Linear",     # Quantize all linear layers (most of the model)
    ignore=["lm_head"],   # Keep output layer precise for better predictions
)
```

- **W4A16**: Aggressive weight compression while keeping activations in FP16 for stability
- **targets="Linear"**: Linear layers contain most parameters, so quantizing them gives the biggest savings
- **ignore=["lm_head"]**: The final output layer maps to vocabulary probabilities - keeping it in FP16 preserves output quality

---
## 3. Quantize the Model

Now let's apply our recipe to quantize `Qwen/Qwen2-0.5B-Instruct`.

In [ ]:
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

recipe = [
    GPTQModifier(
        scheme="W4A16",      # or "INT4" / "W4A8" depending on your version
        targets="Linear",
        ignore=["lm_head"],
        # optional: group_size=128, act_order=True, etc.,
        # depending on what llmcompressor exposes
    ),
]

oneshot(
    model=MODEL_ID,
    dataset="wikitext",
    dataset_config_name="wikitext-2-raw-v1",
    recipe=recipe,
    output_dir=OUTPUT_DIR,
    max_seq_length=64,
    num_calibration_samples=4,
)

print(f"\nQuantized model saved to: {OUTPUT_DIR}")

---
## 4. Compare File Sizes

Let's compare the size of the quantized model to the original.

In [13]:
def folder_size(path: str) -> int:
    """Get total size of files in a directory"""
    p = pathlib.Path(path)
    if not p.exists():
        return 0
    return sum(f.stat().st_size for f in p.rglob("*") if f.is_file())

def format_size(bytes: int) -> str:
    """Format bytes as human-readable string"""
    if bytes < 1024:
        return f"{bytes} B"
    elif bytes < 1024**2:
        return f"{bytes/1024:.1f} KB"
    elif bytes < 1024**3:
        return f"{bytes/1024**2:.1f} MB"
    return f"{bytes/1024**3:.2f} GB"

In [ ]:
# Get quantized model size
size_q = folder_size(OUTPUT_DIR)

print("Model Size Comparison")
print("=" * 40)
print(f"Original (FP16):    ~1.0 GB")
print(f"Quantized (W8A8):   {format_size(size_q)}")

In [ ]:
# List the quantized model files
!ls -lh {OUTPUT_DIR}/

---
## 5. Test the Models

Let's run a quick inference test to compare the original and quantized models.

In [ ]:
# Load the original model
from transformers import AutoModelForCausalLM, AutoTokenizer

original_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
original_model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="cpu")

# Test prompt using chat template for proper formatting
prompt = "Complete this sentence: Model compression is good because"
messages = [{"role": "user", "content": prompt}]
inputs = original_tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
outputs = original_model.generate(inputs, max_new_tokens=100, do_sample=True, temperature=0.7, pad_token_id=original_tokenizer.eos_token_id)

# Decode only the generated portion (skip the input tokens)
generated_tokens = outputs[0][inputs.shape[-1]:]
print("Original Model")
print("Prompt:", prompt)
print("Response:", original_tokenizer.decode(generated_tokens, skip_special_tokens=True))

In [ ]:
# Load the quantized model
import logging
logging.getLogger("llmcompressor").setLevel(logging.WARNING)

quantized_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
quantized_model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR, device_map="cpu")

# Test with same prompt
inputs = quantized_tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
outputs = quantized_model.generate(inputs, max_new_tokens=100, do_sample=True, temperature=0.7, pad_token_id=quantized_tokenizer.eos_token_id)

# Decode only the generated portion (skip the input tokens)
generated_tokens = outputs[0][inputs.shape[-1]:]
print("Quantized Model")
print("Prompt:", prompt)
print("Response:", quantized_tokenizer.decode(generated_tokens, skip_special_tokens=True))

---
## 6. Perplexity Comparison

Perplexity measures how well a model predicts text - lower is better. Let's compare the original and quantized models.

In [ ]:
import math
import torch
from datasets import load_dataset

def calculate_perplexity(model, tokenizer, dataset, max_tokens=5000, stride=512):
    """Calculate perplexity using sliding window evaluation."""
    # Tokenize entire dataset as one long sequence
    encodings = tokenizer("\n\n".join(dataset["text"]), return_tensors="pt")
    input_ids = encodings.input_ids[:, :max_tokens]
    
    nlls = []
    prev_end = 0
    
    for begin_loc in range(0, input_ids.size(1), stride):
        end_loc = min(begin_loc + stride, input_ids.size(1))
        trg_len = end_loc - prev_end
        input_slice = input_ids[:, begin_loc:end_loc]
        target_slice = input_slice.clone()
        target_slice[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_slice, labels=target_slice)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)
        prev_end = end_loc

    return math.exp(torch.stack(nlls).sum() / prev_end)

# Load test dataset
test_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

In [ ]:
# Calculate perplexity for original model
original_ppl = calculate_perplexity(original_model, original_tokenizer, test_dataset)
print(f"Original Model Perplexity: {original_ppl:.2f}")

In [ ]:
# Calculate perplexity for quantized model
quantized_ppl = calculate_perplexity(quantized_model, quantized_tokenizer, test_dataset)
print(f"Quantized Model Perplexity: {quantized_ppl:.2f}")

In [ ]:
# Compare results
print("Perplexity Comparison")
print("=" * 40)
print(f"Original Model:   {original_ppl:.2f}")
print(f"Quantized Model:  {quantized_ppl:.2f}")
print(f"Difference:       {quantized_ppl - original_ppl:+.2f} ({(quantized_ppl/original_ppl - 1)*100:+.1f}%)")

---
## Summary

You've learned:

✅ The `oneshot` API is llm-compressor's main interface for quantization

✅ `GPTQModifier` configures the quantization algorithm and scheme

✅ Key parameters: `scheme`, `targets`, `ignore`, `group_size`

✅ Quantization significantly reduces model size

✅ Quantized models produce comparable outputs to the original

✅ Perplexity testing measures the quality impact of quantization

**Next:** Continue to [Advanced Quantization](./2-advanced-quantization.ipynb) to explore different schemes and compare trade-offs.